In [1]:
import optuna
import numpy as np
import pandas as pd
from cleanlab import Datalab

optuna.logging.set_verbosity(optuna.logging.WARNING)

In [2]:
from sklearnex import patch_sklearn

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [4]:
import fasttext
from tqdm.auto import tqdm

encoder = fasttext.load_model("./encoders/cc.ar.300.bin")

In [5]:
dataset = pd.read_excel("./data/to_label.xlsx", sheet_name="Sheet1")
dataset["label"] = dataset["label"].astype(int)
dataset.head()

,tweetid,tweet_text,clean_tweet,tech,label
0,295735762505322497,النعيم أمي، وعين أمي، وقبلة أمي، فأكرمها يالله...,النعيم أمي، وعين أمي، وقبلة أمي، فأكرمها يالله...,NaN,0
1,1080873494227886086,RT @rood516: عرض الاجازه😍👌🏻1\nســــاعه\nقـــلم...,RT : عرض الاجازه1 ســــاعه قـــلم كــبك نظـــا...,NaN,0
2,1027481525506453504,RT @6FSH1_: مهما ضاق صدرك تذكر أن في ناس توهم ...,RT : مهما ضاق صدرك تذكر أن في ناس توهم بيسجلون...,NaN,0
3,417202038037299200,RT @f450450: ﴿ياحي ياقيوم برحمتك أستغيث أصلح ل...,RT : ﴿ياحي ياقيوم برحمتك أستغيث أصلح لي شأني ك...,NaN,0
4,834074569673736193,RT @7744Qa: بعض البشر ، إذا وجد البديل نكر الج...,RT : بعض البشر ، إذا وجد البديل نكر الجميل. قر...,NaN,0


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweetid      2100 non-null   int64 
 1   tweet_text   2100 non-null   object
 2   clean_tweet  2100 non-null   object
 3   tech         202 non-null    object
 4   label        2100 non-null   int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 74.0+ KB


In [7]:
tweets = dataset.clean_tweet.to_list()
labels = dataset.label.values

In [8]:
features = []

for tweet in tqdm(tweets):
    features.append(encoder.get_sentence_vector(tweet))

  0%|          | 0/2100 [00:00<?, ?it/s]

In [9]:
data = {"X": np.array(features), "Y": labels}

In [10]:
w = dataset.label.value_counts(normalize=True).to_list()
w = {i: w[i] for i in range(len(w))}
w

{0: 0.9038095238095238, 1: 0.09619047619047619}

In [11]:
x_train, x_test, y_train, y_test = train_test_split(
    data["X"], data["Y"], test_size=0.25, stratify=data["Y"], random_state=42
)

In [12]:
def objective(trial):
    p = trial.suggest_categorical("penalty", ["l1", "l2"])
    s = trial.suggest_categorical("solver", ["liblinear"])
    c = trial.suggest_categorical("C", [0.1, 1.0, 10.0, 100.0, 1000.0])
    model = LogisticRegression(
        penalty=p, solver=s, C=c, class_weight=w, max_iter=1000, random_state=42
    )
    model.fit(x_train, y_train)
    # acc = model.score(x_test, y_test)
    f1 = f1_score(y_test, model.predict(x_test), average="binary")
    return f1

In [13]:
study = optuna.create_study(
    study_name="propaganda cleaning",
    direction="maximize",
    storage="sqlite:///propaganda.db",
    load_if_exists=False,
)

study.optimize(objective, n_trials=1000, n_jobs=-1, show_progress_bar=True)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
study.best_params

{'penalty': 'l1', 'solver': 'liblinear', 'C': 1000.0}

In [15]:
model = LogisticRegression(
    class_weight=w, random_state=42, max_iter=1000, **study.best_params
)
pred_probs = cross_val_predict(
    estimator=model, X=data["X"], y=data["Y"], cv=50, method="predict_proba"
)

In [16]:
lab = Datalab(data, label_name="Y")
lab.find_issues(pred_probs=pred_probs, features=data["X"])

Finding label issues ...
Finding outlier issues ...
Fitting OOD estimator based on provided features ...
Finding near_duplicate issues ...
Audit complete. 296 issues found in the dataset.


In [17]:
lab.report()

Here is a summary of the different kinds of issues found in the data:

    issue_type  num_issues
         label         209
near_duplicate          46
       outlier          41

Dataset Information: num_examples: 2100, num_classes: 2


----------------------- label issues -----------------------

About this issue:
	Examples whose given label is estimated to be potentially incorrect
    (e.g. due to annotation error) are flagged as having label issues.
    

Number of examples with this issue: 209
Overall dataset quality in terms of this issue: 0.8914

Examples representing most severe instances of this issue:
      is_label_issue   label_score  given_label  predicted_label
1715            True  0.000000e+00            0                1
502             True  0.000000e+00            0                1
284             True  0.000000e+00            0                1
2028            True  0.000000e+00            0                1
373             True  2.920535e-13            1         

In [18]:
pd.set_option("display.max_colwidth", None, "display.max_rows", None)

In [19]:
issues = lab.get_issues()

In [20]:
issues.head()

,is_label_issue,label_score,is_outlier_issue,outlier_score,is_near_duplicate_issue,near_duplicate_score
0,False,0.999725,False,0.720818,False,0.272238
1,False,1.000000,False,0.808768,False,0.178953
2,False,0.999028,False,0.757022,False,0.259652
3,False,0.999994,False,0.785526,False,0.128088
4,False,1.000000,False,0.854087,False,0.120545


In [21]:
label_issues = dataset[issues.is_label_issue]
label_issues.info()

<class 'pandas.core.frame.DataFrame'>
Index: 209 entries, 14 to 2087
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweetid      209 non-null    int64 
 1   tweet_text   209 non-null    object
 2   clean_tweet  209 non-null    object
 3   tech         116 non-null    object
 4   label        209 non-null    int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 9.0+ KB


In [22]:
label_issues.head(100)

,tweetid,tweet_text,clean_tweet,tech,label
14,691276323428282368,كوريا اكتشافي بدا الكل يمدحه .. روووعة,كوريا اكتشافي بدا الكل يمدحه .. روووعة,loaded language - name-calling,1
28,851516912924340224,سجل يا تاريخ واحفظها يا تاريخ النصر\nلاول مرة منذ تأسيس نادينا\nاصبحنا كوشه لنادي الهلال\nبسبب فكر رئيس مايعرف من كرة القدم\nالا ميسي وحسين 😂,سجل يا تاريخ واحفظها يا تاريخ النصر لاول مرة منذ تأسيس نادينا اصبحنا كوشه لنادي الهلال بسبب فكر رئيس مايعرف من كرة القدم الا ميسي وحسين,flag-waving - name-calling - smears,1
34,1137141663397269505,RT @ryadygadeem: #AlhilalwantsFifaToSavehim3\nونستمر في هذي الحمله المليونيه لأجل الهلال💪 https://t.co/T434ilELwS,RT : AlhilalwantsFifaToSavehim3 ونستمر في هذي الحمله المليونيه لأجل الهلال,NaN,0
41,1052603026425487360,RT @yi4_5: #المصمم_دافي_العتيبي_مبدع\n#همس_الغلا_مصممه_مبدعه\n\nيستاهلون الترند سما سما 🚀🚀🚀 https://t.co/fgIOXxqJQY,RT : المصمم دافي العتيبي مبدع همس الغلا مصممه مبدعه يستاهلون الترند سما سما,NaN,0
65,1098592460069105664,راشد الماجد\n#سايحون_رررغم_الطناطير,راشد الماجد سايحون رررغم الطناطير,NaN,0
68,460142383833681920,شرطة منطقة #الرياض توقع بعصابة سرقة #السيارات تحت تهديد السلاح مؤلفة من ثلاثة أشخاص,شرطة منطقة الرياض توقع بعصابة سرقة السيارات تحت تهديد السلاح مؤلفة من ثلاثة أشخاص,NaN,0
102,784479049053827072,RT @iMUTEB4: الاب يصنع المستحيل في اقسى الظروف من اجل أطفاله .. https://t.co/1KK6A3K41h,RT : الاب يصنع المستحيل في اقسى الظروف من اجل أطفاله ..,NaN,0
125,1020854328804937730,ربي آغفرلي وآرحمني\n#كلنا_مع_عبدالعزيز_الجبرين,ربي آغفرلي وآرحمني كلنا مع عبدالعزيز الجبرين,flag-waving,1
133,981562439757189126,RT @Asslsdr3Asslsdr: #الشرع_محلل_اربع\n#كلنا_المريسل \n#المدينة_المنورة #مكة_المكرمة #جده #ينبع_الصناعية #ينبع #تبوك \nلمن يبحث عن جودة العسل…,RT : الشرع محلل اربع كلنا المريسل المدينة المنورة مكة المكرمة جده ينبع الصناعية ينبع تبوك لمن يبحث عن جودة العسل…,flag-waving,1
135,656822170132176896,RT @TAIZ_NOW: #كاريكاتير .. المسيرة الشيطانية مرت من هنا ! https://t.co/oqLbhIarBd,RT : كاريكاتير .. المسيرة الشيطانية مرت من هنا !,name-calling - loaded language,1


In [23]:
df = dataset[["tweetid", "tweet_text", "clean_tweet", "tech", "label"]].copy()
df = df.rename(columns={"clean_tweet": "text"})
df.head()

,tweetid,tweet_text,text,tech,label
0,295735762505322497,النعيم أمي، وعين أمي، وقبلة أمي، فأكرمها يالله بطول العمر بالصحه والعافية وازرع في قلبها سعادة لاتزول ♥,النعيم أمي، وعين أمي، وقبلة أمي، فأكرمها يالله بطول العمر بالصحه والعافية وازرع في قلبها سعادة لاتزول,NaN,0
1,1080873494227886086,RT @rood516: عرض الاجازه😍👌🏻1\nســــاعه\nقـــلم\nكــبك\nنظـــاره\nسبــحه 📿\nمـــيداليه\nخاتم\nمحفظه\nضمان الساعه لمدة سنه\nفقط ٢٢٠ريال\n🌟🌟\nواتس للطلب ا…,RT : عرض الاجازه1 ســــاعه قـــلم كــبك نظـــاره سبــحه مـــيداليه خاتم محفظه ضمان الساعه لمدة سنه فقط ٢٢٠ريال واتس للطلب ا…,NaN,0
2,1027481525506453504,RT @6FSH1_: مهما ضاق صدرك تذكر أن في ناس توهم بيسجلون بالجامعة.,RT : مهما ضاق صدرك تذكر أن في ناس توهم بيسجلون بالجامعة.,NaN,0
3,417202038037299200,RT @f450450: ﴿ياحي ياقيوم برحمتك أستغيث أصلح لي شأني كُلہ ولا تَكِلْني إلى نفسي طرفہ عين﴾,RT : ﴿ياحي ياقيوم برحمتك أستغيث أصلح لي شأني كلہ ولا تكلني إلى نفسي طرفہ عين﴾,NaN,0
4,834074569673736193,RT @7744Qa: بعض البشر ، إذا وجد البديل نكر الجميل.\n#قروب_شوشي_للدعم \n#قروب_همس_المشاعر \n#قروب_سججات_بدوي \n#قروب_السمو_للدعم,RT : بعض البشر ، إذا وجد البديل نكر الجميل. قروب شوشي للدعم قروب همس المشاعر قروب سججات بدوي قروب السمو للدعم,NaN,0


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetid     2100 non-null   int64 
 1   tweet_text  2100 non-null   object
 2   text        2100 non-null   object
 3   tech        202 non-null    object
 4   label       2100 non-null   int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 74.0+ KB


In [25]:
# df.to_json("./data/to_label.json")